In [5]:
import pandas as pd

In [2]:
areas = ['City of London', 'Barking and Dagenham', 'Barnet', 'Bexley', 'Brent', 'Bromley', 'Camden', 'Croydon', 'Ealing', 'Enfield', 'Greenwich', 'Hackney', 'Hammersmith and Fulham', 'Haringey', 'Harrow', 'Havering', 'Hillingdon', 'Hounslow', 'Islington', 'Kensington and Chelsea', 'Kingston upon Thames', 'Lambeth', 'Lewisham', 'Merton', 'Newham', 'Redbridge', 'Richmond upon Thames', 'Southwark', 'Sutton', 'Tower Hamlets', 'Waltham Forest', 'Wandsworth', 'Westminster']

## Housing density

In [10]:
housing_density = pd.read_excel('data/housing density.xlsx', sheet_name= 'Number of dwellings')
housing_density = housing_density.drop(index=0)
for area_london in housing_density['Area name']:
    if area_london not in areas:
        housing_density = housing_density[housing_density['Area name'] != area_london]
housing_density.rename(columns={'Area name': 'Area'}, inplace=True)
housing_density.rename(columns={'ONS code': 'Code'}, inplace=True)

# TODO: actually compute the density --> waiting for mischa's data
housing_density

,Code,Area,2001,2002,2003,2004,2005,2006,2007,2008,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,E09000001,City of London,5009.0,5117.100000,5126.200000,5263.300000,5374.400000,5373.500000,5374.600000,5420.700000,...,5458.900000,5513.0,5531.000000,5566.000000,6003.000000,6229.000000,6306.000000,6313.000000,6451.000000,6506.000000
2,E09000002,Barking and Dagenham,68298.0,68525.700000,68837.400000,68899.100000,69260.800000,69528.500000,69835.200000,70550.900000,...,70946.300000,71079.0,71431.000000,71937.000000,72668.000000,73182.000000,73914.000000,74510.000000,74923.000000,75829.000000
3,E09000003,Barnet,130515.0,130800.600000,131883.200000,132855.800000,133902.400000,134941.000000,135428.600000,136680.200000,...,138616.400000,139346.0,141460.645482,142834.645482,143947.645482,145271.645482,146729.645482,148528.645482,150736.645482,152945.645482
4,E09000004,Bexley,91606.0,91907.600000,92223.200000,92949.800000,93220.400000,93433.000000,93745.600000,94078.200000,...,94798.400000,95037.0,95240.000000,95658.000000,96186.000000,96996.000000,96864.000000,97628.000000,97905.000000,98391.000000
5,E09000005,Brent,101427.0,102828.100000,103846.200000,104454.300000,105093.400000,106767.500000,107969.600000,109036.700000,...,111413.900000,112083.0,112643.000000,113305.000000,114039.000000,115598.000000,116649.000000,118013.000000,118707.000000,120448.000000
6,E09000006,Bromley,128717.0,128974.200000,129314.400000,129808.600000,130639.800000,131387.000000,132339.200000,133112.400000,...,134297.800000,135036.0,135612.000000,136304.000000,136453.000000,136864.000000,137564.000000,138422.000000,138976.000000,139684.000000
7,E09000007,Camden,93112.0,93884.100000,94460.200000,94841.300000,95563.400000,96350.500000,96997.600000,97516.700000,...,99138.900000,99828.0,100196.000000,100760.000000,101214.000000,101653.000000,102618.000000,103826.000000,104771.000000,105598.000000
8,E09000008,Croydon,141300.0,141423.600000,141819.200000,142176.800000,142566.400000,142961.000000,143713.600000,144896.200000,...,147249.400000,148099.0,148806.000000,149697.000000,150992.000000,152515.000000,154559.000000,157394.000000,159470.000000,161060.000000
9,E09000009,Ealing,120331.0,120589.700000,120922.400000,121428.100000,121909.800000,122743.500000,124111.200000,125530.900000,...,126901.300000,127187.0,127872.000000,128862.000000,129631.000000,130528.000000,131249.000000,132094.000000,133551.000000,135305.000000
10,E09000010,Enfield,112948.0,114130.900000,115273.800000,116337.700000,117055.600000,117613.500000,118667.400000,119946.300000,...,121243.100000,122042.0,122339.000000,122888.000000,123400.000000,123799.000000,124471.000000,125369.000000,125755.000000,126255.000000


## Crime

In [26]:
crime = pd.read_csv('data/crime.csv')
crime = crime.drop(crime.columns[[0,1]], axis=1)
crime = crime.reset_index(drop=True)
for col in crime.columns[1:]:
    if '04' in col:
        continue
    else:
        crime = crime.drop(col, axis=1)

crime = crime.groupby('LookUp_BoroughName').agg('sum').reset_index()

for area_london in crime['LookUp_BoroughName']:
    if area_london not in areas:
        crime = crime[crime['LookUp_BoroughName'] != area_london]

new_name = {}
for col in crime.columns[1:]:
    new_name[col] = col[:4]

crime.rename(columns={'LookUp_BoroughName': 'Area'}, inplace=True)
crime.rename(columns=new_name, inplace=True)
crime = crime.dropna()

# Adding missing Area Column
crime = pd.merge(housing_density[['Code', 'Area']],crime, on = 'Area')

# save clean data
crime.to_csv('clean_data/crime.csv')

crime

,Code,Area,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,E09000002,Barking and Dagenham,1669,1635,1363,1360,1253,1343,1414,1497,1454,1684,1273
1,E09000003,Barnet,2037,2243,2098,1817,1917,1925,2168,2207,2352,2557,1660
2,E09000004,Bexley,1214,982,984,940,1055,918,966,1093,1194,1488,916
3,E09000005,Brent,2491,2601,2253,1851,2017,2021,2217,2323,2400,2447,2036
4,E09000006,Bromley,1887,1797,1520,1646,1583,1725,1584,1914,1890,1968,1444
5,E09000007,Camden,2903,2990,2759,2404,2181,2516,2314,2916,2831,3235,1643
6,E09000008,Croydon,2708,2625,2566,2250,2121,2399,2458,2395,2510,2621,2251
7,E09000009,Ealing,2986,2915,2400,2195,1972,2109,2338,2254,2291,2506,2000
8,E09000010,Enfield,1933,1910,1802,1730,1747,1771,1942,2096,2312,2416,1867
9,E09000011,Greenwich,2155,1861,1674,1628,1646,1740,1871,2049,2094,2353,1663


## Waste

In [28]:
waste = pd.read_excel('data/recycle.xlsx', sheet_name='Household Recycling Rates')
waste = waste.drop(index=0)
waste = waste.drop('Code', axis=1)
for area_london in waste['Area']:
    if area_london not in areas:
        waste = waste[waste['Area'] != area_london]
new_name = {}
for col in waste.columns[2:]:
    new_name[col] = col[:4]

waste.rename(columns={'New Code': 'Code'}, inplace=True)
waste.rename(columns=new_name).dropna()
waste.rename(columns=new_name, inplace=True)
waste = waste.dropna()

# save clean data
waste.to_csv('clean_data/waste.csv')

waste

,Code,Area,1998,1999,2000,2001,2002,2003,2004,2005,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
1,E09000001,City of London,0.0,0,0.0,0.0,1.0,8.0,14.0,18.10,...,36.9,36.4,38.8,34.4,30.8,28.5,29.236600,29.9,28.9,30.3
2,E09000002,Barking and Dagenham,3.0,3,4.0,3.0,3.0,5.0,14.0,16.60,...,30.0,26.8,24.8,23.4,18.9,25.3,25.046448,23.7,25.2,26.9
3,E09000003,Barnet,9.0,8,8.0,8.0,13.0,16.0,20.0,27.47,...,33.6,33.0,36.4,38.0,36.8,37.4,36.928207,34.6,32.0,28.9
4,E09000004,Bexley,20.0,18,17.0,20.0,23.0,21.0,30.0,37.71,...,53.5,54.3,55.2,54.0,52.0,52.7,52.061997,54.1,54.2,50.0
5,E09000005,Brent,5.0,6,7.0,7.0,8.0,10.0,14.0,20.01,...,36.8,42.5,40.8,35.2,35.8,36.4,36.534757,36.6,34.2,33.4
6,E09000006,Bromley,7.0,13,13.0,14.0,15.0,20.0,23.0,27.25,...,49.9,49.1,49.6,48.0,46.3,46.9,49.987760,50.1,50.9,44.8
7,E09000007,Camden,11.0,13,16.0,12.0,14.0,19.0,25.0,27.14,...,33.0,30.9,29.3,26.3,24.8,26.6,30.328873,31.1,25.9,28.6
8,E09000008,Croydon,14.0,13,12.0,12.0,13.0,14.0,13.0,16.17,...,38.1,44.3,42.2,39.9,37.8,38.6,37.915908,47.3,49.2,41.3
9,E09000009,Ealing,10.0,12,12.0,13.0,15.0,15.0,15.0,19.28,...,40.8,40.6,40.2,40.1,43.0,50.7,48.761060,52.6,48.3,49.0
10,E09000010,Enfield,9.0,9,8.0,13.0,14.0,16.0,24.0,27.29,...,35.3,38.8,39.1,38.5,35.9,37.2,35.873427,33.4,33.1,33.2
